# AutoProposal – Client Proposal Generator Agent (Kaggle Demo)
This notebook builds a simple multi-agent pipeline that:
- Parses a client brief
- Researches competitors (DuckDuckGo search)
- Estimates pricing & ROI (mock model)
- Generates a proposal PDF

**Run each code cell in order (Shift+Enter).**  
If you don't have API keys, the notebook runs in **MOCK MODE** (no external APIs).

In [1]:
# Install required packages (run once)
!pip install --quiet openai==1.40.2 reportlab duckduckgo-search python-dotenv
print("✔︎ Dependencies installed")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 54.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
litellm 1.76.3 requires openai>=1.99.5, but you have openai 1.40.2 which is incompatible.
✔︎ Dependencies installed


In [2]:
# Imports and environment loader
import os
import json
import random
from duckduckgo_search import DDGS
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from dotenv import load_dotenv

load_dotenv()  # loads .env if present

# Check for OpenAI key (optional)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_API_KEY:
    print("OpenAI key found — real LLM mode enabled (optional).")
else:
    print("🔄 MOCK MODE: No OpenAI key found. LLM calls will be mocked.")

🔄 MOCK MODE: No OpenAI key found. LLM calls will be mocked.


In [3]:
# Simple LLM wrapper: uses OpenAI if key present, otherwise returns mock text
def llm(prompt):
    """
    If OPENAI_API_KEY present, attempt a Chat completion.
    Otherwise return a deterministic MOCK response.
    """
    if OPENAI_API_KEY is None:
        # deterministic mock: keep output short but useful
        sample = (
            "MOCK SUMMARY: "
            + prompt.split("\n")[0][:150]
            + " ...\n\n(Use real API key to enable full LLM responses.)"
        )
        return sample

    # Real API usage (if you add key later)
    try:
        from openai import OpenAI
        client = OpenAI(api_key=OPENAI_API_KEY)
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role":"user","content":prompt}],
            max_tokens=600
        )
        return completion.choices[0].message.content
    except Exception as e:
        return f"LLM ERROR (falling back to MOCK): {e}"

In [4]:
def competitor_research(industry, keywords="", max_results=5):
    """
    Use DuckDuckGo to fetch top pages for competitor research.
    Returns list of title & url strings. In mock mode returns sample list.
    """
    print("🔍 Running competitor research...")
    try:
        results = []
        with DDGS() as ddgs:
            query = f"{industry} {keywords} competitors"
            for r in ddgs.text(query, max_results=max_results):
                # r has 'title' and 'href'
                title = r.get("title", "")[:200]
                href = r.get("href", "")
                results.append(f"{title} - {href}")
        if not results:
            raise Exception("No results")
        return results
    except Exception as e:
        print("⚠️ Research tool fallback (mock).", e)
        # Mock list
        return [
            f"{industry} Competitor A - https://example.com/a",
            f"{industry} Competitor B - https://example.com/b",
            f"{industry} Competitor C - https://example.com/c",
        ]


In [5]:
def calculate_pricing(deliverables_count, complexity=1.5, base_price=150):
    """
    Simple pricing model:
      price = base_price * complexity * deliverables_count
    ROI: random multiplier to simulate business value
    """
    price = base_price * complexity * max(1, int(deliverables_count))
    roi_multiplier = random.uniform(2.0, 4.0)
    roi_value = round(price * roi_multiplier, 2)
    return {
        "estimated_price": round(price, 2),
        "expected_roi_value": roi_value,
        "details": {
            "base_price": base_price,
            "complexity": complexity,
            "deliverable_count": deliverables_count
        }
    }


In [6]:
def generate_pdf(client_name, proposal_text, out_dir="/kaggle/working"):
    """
    Very simple PDF generator using reportlab.
    Saves PDF to Kaggle working directory for easy download.
    """
    safe_name = client_name.replace(" ", "_")
    file = os.path.join(out_dir, f"{safe_name}_proposal.pdf")
    c = canvas.Canvas(file, pagesize=A4)
    width, height = A4

    # Simple header
    c.setFont("Helvetica-Bold", 16)
    c.drawString(40, height - 60, f"Proposal for {client_name}")

    # Body text
    text = c.beginText(40, height - 100)
    text.setFont("Helvetica", 10)
    max_width_chars = 110
    for paragraph in proposal_text.split("\n"):
        # naive wrap
        while len(paragraph) > max_width_chars:
            text.textLine(paragraph[:max_width_chars])
            paragraph = paragraph[max_width_chars:]
        text.textLine(paragraph)
        text.textLine("")  # blank line

    c.drawText(text)
    c.showPage()
    c.save()
    print(f"📄 PDF generated at: {file}")
    return file


In [7]:
def requirements_agent(client_brief):
    """
    Use LLM (or mock) to extract structured fields from the brief.
    Returns dict with industry, goals, pain_points, deliverables (list), audience.
    """
    prompt = f"""
    Extract these items from the client brief (as JSON):
    - industry
    - goals
    - pain_points
    - deliverables (list; up to 7 items)
    - target_audience

    Client brief:
    {client_brief}

    Return JSON only.
    """
    raw = llm(prompt)
    # Try to parse JSON from LLM; if mock, create fallback structure
    try:
        # Attempt to find first '{' for JSON
        start = raw.find("{")
        if start != -1:
            data = json.loads(raw[start:])
            return data
    except Exception:
        pass

    # Fallback (mock)
    return {
        "industry": "Digital Marketing",
        "goals": "Increase leads and brand awareness by 50% in 6 months",
        "pain_points": "Low lead volume; inconsistent messaging",
        "deliverables": ["SEO optimization", "PPC campaigns", "Landing page", "Email automation"],
        "target_audience": "SMB owners and marketing managers"
    }


In [8]:
def research_agent(industry):
    competitors = competitor_research(industry, keywords="digital marketing")
    # Summarize using LLM (mock if no key)
    summary_prompt = f"Summarize key insights from these competitors:\n\n{json.dumps(competitors, indent=2)}\n\nGive 5 bullet points."
    summary = llm(summary_prompt)
    return {"summary": summary, "competitors": competitors}


In [9]:
def pricing_agent(deliverables):
    count = len(deliverables)
    # complexity estimate heuristic: longer deliverable names => higher complexity
    avg_len = sum(len(d) for d in deliverables) / max(1, count)
    complexity = 1.0 + min(2.0, avg_len / 30.0)
    pricing = calculate_pricing(count, complexity=complexity)
    return pricing


In [10]:
def proposal_writer(parsed, research, pricing):
    """
    Create a professional proposal body (string).
    """
    prompt = f"""
    Write a professional B2B proposal using the fields below.
    Include: Executive summary, Proposed Scope (with deliverables),
    Timeline (3 milestones), Pricing summary, ROI estimate, Risks & Next steps.
    
    Parsed Info:
    {json.dumps(parsed, indent=2)}

    Research Summary:
    {research['summary']}

    Competitors:
    {json.dumps(research['competitors'], indent=2)}

    Pricing:
    {json.dumps(pricing, indent=2)}
    """
    proposal_text = llm(prompt)
    # If mock output is short, augment with template
    if proposal_text.startswith("MOCK"):
        # naive template build
        lines = []
        lines.append(f"Executive Summary\nWe propose a {parsed['industry']} engagement to achieve: {parsed['goals']}.")
        lines.append("\nProposed Scope:")
        for d in parsed['deliverables']:
            lines.append(f"- {d}")
        lines.append("\nTimeline:")
        lines.append("1. Discovery (2 weeks)")
        lines.append("2. Implementation (8 weeks)")
        lines.append("3. Optimization (4 weeks)")
        lines.append("\nPricing Summary:")
        lines.append(f"Total estimated price: ${pricing['estimated_price']}")
        lines.append(f"Expected ROI (estimated): ${pricing['expected_roi_value']}")
        lines.append("\nRisks & Next Steps:")
        lines.append("- Risk: availability of client assets")
        lines.append("- Next step: sign SOW and schedule kickoff")
        proposal_text = "\n".join(lines)
    return proposal_text


In [11]:
def coordinator(client_brief, client_name="Client"):
    print("=== STEP 1: Parse requirements ===")
    parsed = requirements_agent(client_brief)
    print("Parsed:", parsed)

    print("\n=== STEP 2: Research competitors ===")
    research = research_agent(parsed.get("industry", ""))
    print("Research summary (truncated):", research['summary'][:300])

    print("\n=== STEP 3: Pricing & ROI ===")
    pricing = pricing_agent(parsed.get("deliverables", []))
    print("Pricing:", pricing)

    print("\n=== STEP 4: Write proposal ===")
    proposal_text = proposal_writer(parsed, research, pricing)
    print("Proposal length:", len(proposal_text))

    print("\n=== STEP 5: Generate PDF ===")
    pdf_path = generate_pdf(client_name, proposal_text)

    # Return structured result
    return {
        "parsed": parsed,
        "research": research,
        "pricing": pricing,
        "proposal_text": proposal_text,
        "pdf_path": pdf_path
    }


In [12]:
# Example client brief — edit this to try different inputs
client_brief = """
We are BrightShop, an e-commerce startup selling eco-friendly home goods.
Goal: Increase monthly leads by 50% and double online conversion in 6 months.
Budget range: $5,000 - $12,000 for initial 3-month engagement.
We need SEO, Google Ads management, landing page optimization, and email flows.
Target audience: urban 25-45 eco-conscious buyers.
"""

result = coordinator(client_brief, client_name="BrightShop")
result  # shows dict output; PDF path is in result['pdf_path']


=== STEP 1: Parse requirements ===
Parsed: {'industry': 'Digital Marketing', 'goals': 'Increase leads and brand awareness by 50% in 6 months', 'pain_points': 'Low lead volume; inconsistent messaging', 'deliverables': ['SEO optimization', 'PPC campaigns', 'Landing page', 'Email automation'], 'target_audience': 'SMB owners and marketing managers'}

=== STEP 2: Research competitors ===
🔍 Running competitor research...


/tmp/ipykernel_13/386215183.py:9: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


⚠️ Research tool fallback (mock). No results
Research summary (truncated): MOCK SUMMARY: Summarize key insights from these competitors: ...

(Use real API key to enable full LLM responses.)

=== STEP 3: Pricing & ROI ===
Pricing: {'estimated_price': 885.0, 'expected_roi_value': 2259.96, 'details': {'base_price': 150, 'complexity': 1.475, 'deliverable_count': 4}}

=== STEP 4: Write proposal ===
Proposal length: 485

=== STEP 5: Generate PDF ===
📄 PDF generated at: /kaggle/working/BrightShop_proposal.pdf


{'parsed': {'industry': 'Digital Marketing',
  'goals': 'Increase leads and brand awareness by 50% in 6 months',
  'pain_points': 'Low lead volume; inconsistent messaging',
  'deliverables': ['SEO optimization',
   'PPC campaigns',
   'Landing page',
   'Email automation'],
  'target_audience': 'SMB owners and marketing managers'},
 'research': {'summary': 'MOCK SUMMARY: Summarize key insights from these competitors: ...\n\n(Use real API key to enable full LLM responses.)',
  'competitors': ['Digital Marketing Competitor A - https://example.com/a',
   'Digital Marketing Competitor B - https://example.com/b',
   'Digital Marketing Competitor C - https://example.com/c']},
 'pricing': {'estimated_price': 885.0,
  'expected_roi_value': 2259.96,
  'details': {'base_price': 150, 'complexity': 1.475, 'deliverable_count': 4}},
 'proposal_text': 'Executive Summary\nWe propose a Digital Marketing engagement to achieve: Increase leads and brand awareness by 50% in 6 months.\n\nProposed Scope:\n- 